In [ ]:
#-----SETTINGS-----
cif_directory         = "../PathToCifFiles/" #Path to directory containing cifs
output_csv            = "./epoch.csv" 
pressures_to_evaluate = [40,1000,4000] #pressures in Pascal [0-10000]


charge_column         = "" #If left blank, the line containing the first instance of the word 'charge' will be considered the charge column. 

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
def replaceTabs(text:str, spaces = "   ") -> str:
    text = text.strip('\n')
    text = text.strip()
    return text.replace('\t', spaces)


def getChargeListFromCif(cif_address:str,charge_column = "") -> list:
    """
    Take CIF file address: string
    Return list of atom site charges: array of floats
    
    Developed for parsing charges without dependencies on external libraries.
    A charge column can be specified, otherwise it will search for the first instance of the word 'charge' in the CIF. 
    
    In the instance where the word 'charge' occurs prior to the atoms block loop,the function will be unable to return the list of charges. 
    Therefore, it is better to specify the charge column if it is known.
    """
    
    f = open(cif_address, 'r')
    data = f.read()
    f.close()
    
    data = replaceTabs(data)
    if charge_column == "":
        fh = data.split('charge')[0].split('\n')[-1].strip() #first half
        sh = data.split('charge')[1].split('\n')[0].strip() #second half
        charge_column = f'{fh}charge{sh}'
    CHARGE_COLUMN = f"{charge_column}\n"
    atom_charges = []
    if len(data.split(f'{CHARGE_COLUMN}'))  == 1:
        return []
    
    cols_from_end = 1
    block = data.split(f'{CHARGE_COLUMN}')[1]
    lines = block.split('\n')

    for line in lines:
        if line.strip() != "":
            if line.strip()[0] == '_':
                cols_from_end +=1 
            elif line.strip() != "" and len(line.split()) >= 4: 
                col_index = cols_from_end * -1
                charge = line.strip().split()[col_index]
                atom_charges.append(float(charge))
    return atom_charges
                
def get_epoch(Q:float,p:float) -> float:
    """
    Take pressure (p) in Pascal and charge (Q),
    Return estimated Epoch effect (CO2 adsorption)

    Function was estimated by fitting simulations of a non-physical point charge in RASPA. 
    If function estimates the effects to be below 0 (due to the way the data was fit), it will instead return 0.
    
    The function was fit to point charge simulations at/below 10,000Pa; any pressure that is input above this will be reduced to 10,000Pa 
    """
    if p[0] > 10000:
        p = np.ones(len(p)) * 10000
    if p[0]<0:
        return 0
    
    c = np.array([-4.01479419e-01,  9.58680949e-01 , 2.02455722e-01, -3.25777707e-02,-1.44514943e-02 , 5.01944807e-04 , 3.12802362e-04,\
                  1.12292432e-03,-2.00354705e-07  ,1.16669086e-11, -6.28647382e-01]) #coefficients
    var_mat = np.array([Q,Q**2,Q**3,Q**4,Q**5,Q**6,Q**7,p,p**2,p**3,np.ones(len(Q))])
    calc= np.matmul(c,var_mat)
    vals = np.array([max(0,i) for i in calc])
    return sum(vals)

def EpochFromChargeList(clist:list, pressure:float) -> float:
    Q = np.array(clist)
    p = np.ones(len(Q)) * pressure
    return get_epoch(Q,p)


def main():
    cifs         = [os.path.join(cif_directory,i) for i in os.listdir(cif_directory) if os.path.splitext(i)[-1] == '.cif'] #Full paths to cifs
    charges      = {}
    errored_cifs = []

    for cif in cifs:
        try:
            charges[cif] = getChargeListFromCif(cif)
        except:
            errored_cifs.append(cif)
    if len(errored_cifs) >0:
        print('Could not gather charges from:')
        for ec in errored_cifs:
            print(ec)
    else:
        print('Atom site charges gathered.')

    print('\n')
    output = {f'EPoCh_{p}':[] for p in pressures_to_evaluate}
    output['MOF']=[]

    for cif in cifs:
        MOF = os.path.splitext(os.path.split(cif)[-1])[0]
        output['MOF'].append(MOF)
        for pressure in pressures_to_evaluate:
            val = ""
            try:
                val = EpochFromChargeList(charges[cif],pressure)
            except:
                print(f'Unable to gather EPoCh value of {MOF} at {pressure} Pa.')
            output[f'EPoCh_{pressure}'].append(val)


    df = pd.DataFrame(output)
    df = df[['MOF']+[i for i in df.columns[:-1]]]
    df.to_csv(output_csv)
    print('\n\nFinished!')
    print('\n\nPlease Note: Atom-wise, volumetric, or gravimetric averaging of the EPoCh values is reccomended.')
    pass

In [ ]:
main()